# Homework 5 - Visit the Wikipedia hyperlinks graph!

## Claudio Battiloro, Daniele Sanna , Carlo Orsellini

All the useful functions that we need are in the following .py module:


In [ ]:
import hw5_lib as h


We need some others libs:

In [3]:
import pandas as pd
import networkx as nx

In this assignment we perform an analysis of the Wikipedia Hyperlink graph. In particular, given extra information about the categories to which an article belongs to, we are curious to rank the articles according to some criteria.

For this purpose we use the Wikipedia graph released by the SNAP group.


<div style="text-align:center"><img src ="https://www.artribune.com/wp-content/uploads/2016/10/Wikipedia.jpg" /></div>

## [RQ1] Graph Topology

In this section we build the graph $G = (V,E) $ where $V$ is the set of articles and $E$ the hyperlinks among them, and provide its basic information.
We implemented a class $Graph_gen$  to achieve the request which contains all the useful infos.
Let's just instantiate an object by passing to the constructor the path of the -txt file and explore its attributes:

In [4]:
graph = h.Graph_gen(r"C:\Users\claba\Desktop\Python Lavori Congiunti AMDS FDS\AMDS Works\Homework 5\wiki-topcats-reduced.txt")

- Is it directed?

In [15]:
graph.is_dir

True

So we're dealing with a directed graph.

- How many nodes has it?

In [39]:
graph.nodes 

461193

- How many edges has it?

In [19]:
graph.edges

2645247

- What is its average degree?

The average degree is defined as:
$$ \delta = \frac{2|E|}{|V|} $$

where $|E|$ is the number of edges and $|V|$ is the numebr of nodes.

In [23]:
round(graph.avg_degree,3)

11.471

This is a very sparse graph. There are few links w.r.t. the number of nodes. We can check it by computing the graph density that is defined as the number of edges over the maximum possible number of edges:

$$ \rho = \frac{2|E|}{|V|\cdot|V-1|} $$


In [28]:
round(graph.density,6)

2.5e-05

As we expected.

 ## Block Ranking
 
  ![alt text](http://blogg.loppi.se/babymyran/files/2016/07/Lego.png)
     
 

 
Given a category $C_0 = \{article_1,\,article_2,\,\dots \}$ as input we want to rank all of the nodes in V according to the following criteria:

$block_{RANKING} =\begin{bmatrix} C_0 \\ C_1 \\ \dots \\ C_c\\ \end{bmatrix}$

Each category  $C_i$ corresponds  to a list of nodes.

The first category of the rank, $C_0$, always corresponds to the input category. The order of the remaining categories is given by:

$$distance(C_0, C_i) = median(ShortestPath(C_0, C_i))$$

The lower is the distance from $C_0$ the higher is the $C_i$ position in the rank. $ShortestPath(C_0, C_i)$ is the set of all the possible shortest paths between the nodes of $C_0$ and $C_i$. Moreover, the length of a path is given by the sum of the weights of the edges it is composed by that, in this case, we consider unitary.

This measure can create some problems if applied on very sparse graphs as the our becouse it doesn't take in account the very big number of ***not-paths***  that are present. In fact, the most of the nodes are not linked to each others.

To fix this, we decide to add to the formula a term that is proportional to the number of ***not-paths***  in order to improve the ranking of categories that are more linked and less distant from the input category. The high number of missing paths suggested us to use a logarithmic term:

$$ distance(C_0, C_i) = median(ShortestPath(C_0, C_i))\cdot log_{10}(10 + n_{inf}) $$

where $n_{inf}$ is the number of missing links.

To double check the procedure and work with optimized tools, we decide to use also *networkx* package to create and manipulate the graph.

Let's create the net by first reading the DF:


In [5]:

data=pd.read_csv(r'C:\Users\claba\Desktop\Python Lavori Congiunti AMDS FDS\AMDS Works\Homework 5\wiki-topcats-reduced.txt',sep='\t',header=None, engine = "python")
data.columns=['Source','Destination']
data.head()

,Source,Destination
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


Now we can use *networkx* and check its properties :

In [6]:
G=nx.from_pandas_edgelist(data,'Destination','Source',create_using=nx.DiGraph()).to_directed()
nx.info(G)

'Name: \nType: DiGraph\nNumber of nodes: 461193\nNumber of edges: 2645247\nAverage in degree:   5.7357\nAverage out degree:   5.7357'

Now we've to clean up the given categories by cutting the ones that have less than 3500 nodes and filtering all the nodes that are not in the current graph. In order to do this, we implemented a function that takes a nx graph and the path of the cat file and returns a dictionary indexed by cathegory with all the valid nodes:

In [18]:
cat = h.cat_clean(r"C:\Users\claba\Desktop\Python Lavori Congiunti AMDS FDS\AMDS Works\Homework 5\wiki-topcats-categories.txt",G)

Let's see how many categories survived:

In [8]:
len(cat)

35

And their dimension:

In [9]:
cat_df = pd.Series({key: len(value) for key, value in cat.items()}).to_frame()
cat_df.columns = ["Articles"]
cat_df

,Articles
English_footballers,7538
The_Football_League_players,7814
Association_football_forwards,5097
Association_football_goalkeepers,3737
Association_football_midfielders,5827
Association_football_defenders,4588
Living_people,348300
Year_of_birth_unknown,2536
Harvard_University_alumni,5549
Major_League_Baseball_pitchers,5192


Now we need to implement the ranking and we decided to design a ***BFS*** that returns, for each root node, all the reachable nodes with their distance (shortest path) from the root as a  dictionary of the form:

$$\{root : 0,  \{ reachable-node : shortest-path\}\}$$

To do this we implemented a function that takes a graph and a root and returns the aforementioned dict. To speed up the process, for this request, it's better working with our "handmade" graph that is in the form, for each node:
$$\{node : \text{list of destination nodes}: [node_i,... ] \}$$

For example:


In [41]:
graph.graph[62]

[12162, 167659, 279122, 1089199, 1354553, 1400636, 1403619, 1537692, 1544420]

Let's try the BFS:

In [22]:
# The node 52 has the following number of reachable nodes:
len(h.bfs(graph.graph, 52))

301542

The ranking process can be huge, so we decided to implement an inverted index that is indexed by node and contains their categories. This drastically speeds up the process. 

$$ \{ \text{node-number : node-category} \}$$

In [40]:
rev_index = h.cat_inverted_index(cat)

Now we can perform the ranking by using a function that takes the categories dic, the input category name, the inverted index and the graph and returns a dictionary containing tha ranking that we can present in a nicer way using pandas:

In [49]:
rank = h.block_ranking(cat,'English_cricketers',rev_index,graph.graph)
rank_ = pd.DataFrame(rank, columns=['Category','Scores'])
rank_


,Category,Scores
0,English_cricketers,0.000000
1,English_television_actors,15.884298
2,Members_of_the_United_Kingdom_Parliament_for_E...,17.194176
3,Article_Feedback_Pilot,17.591797
4,Fellows_of_the_Royal_Society,18.483759
5,American_Jews,18.785860
6,Black-and-white_films,18.886058
7,American_film_actors,18.893466
8,Indian_films,18.952991
9,American_military_personnel_of_World_War_II,18.995026


## Node Ranking


![image](https://i.pinimg.com/originals/2e/79/d3/2e79d322842b063b0d620b5de334ab1a.gif)


Once we obtain the $block_{RANKING}$ vector, we want to sort the nodes in each category. To do this, we follow the steps indicated:

[**STEP 1**] : Compute subgraph induced by $C_0$. For each node compute the sum of the weigths of the in-edges.

$$score_{article_j} = \sum_{j \in in-edges(article_i)} w_j$$

[**STEP2**] Extend the graph to the nodes that belong to $C_1$. Thus, for each article in $C_1$ compute the score as before. Note that the in-edges coming from the previous category, $C_0$, have as weights the score of the node that sends the edge.

[**STEP3**] Repeat Step2 up to the last category of the ranking.

This 3 steps are implemented in a function that takes the categories dic, the graph and the categories names ordere by block ranking and returns the node ranking as a dictionary of the form:

$$ \{ category : \{ node : weights-sum \} \} $$

Where the categories are in the order of the block ranking.


In [51]:
importlib.reload(h)
node_rank = h.node_ranking(cat,G, rank_.Category)

May be interesting to see which are the articles that obtained the highest overall scores.
To this we implemented a function that takes the *node_rank* dic and the path of the article names file and return a dataframe containing the overall ranking:

In [105]:
name_ = h.name_matching(node_rank, r"C:\Users\claba\Desktop\Python Lavori Congiunti AMDS FDS\AMDS Works\Homework 5\wiki-topcats-page-names.txt" )
name_.head()

,Article,Scores
0,Johnny Weir,10632
1,Jean-Claude Merlin,10578
2,Frank Shu,10576
3,Ora Lassila,10572
4,Oliver Morton (science writer),10572


# Appendix: Signal Processing on Graph

In this small Appendix we wanna suggest some tools that can be very useful to go deeper in this kind of analysis and that come from other disciplinar sectors, like signal processing. 

Nowadays, applications such as social, energy, transportation, sensor,and neuronal networks, high-dimensional data and ,of course, financial data naturally reside on the vertices of weighted graphs. The emerging field of [signal processing on graphs](https://arxiv.org/abs/1211.0053) merges algebraic and spectral graph theoretic concepts with computational harmonic analysis to process such signals on graphs.

We don't want to introduce the math behind this interesting theory but, just for completeness in the definition, a signal on a graph is defined as a *map* from the set $V$ of nodes into the set of complex numbers $\mathbb{C}$:
$$\mathbb{s}: V \rightarrow \mathbb{C}$$
$$\mathbf{s} = [s_1,...,s_{|V|}]^T$$

each element $s_n$ being indexed by node $n$.

It's possible to define signals also on the edges (or on nodes tripletes) and they can be very useful to describe epidemology and (discrete) diffusion in huge networks.

Otherwise, using the [sampling theory for signals on graph](https://arxiv.org/abs/1503.05432) we can try to reconstruct (or predict) missing data by knowing the graph and the values of a finite number of other components ( we can reconstruct a full signal by an its subsampled version) and this can be done by using only the current day data or an entire time series . This procedure are guaranteed and can be extended to more sophisticated application (more on, just to mention one active researcher, [Gonzalo Mateos](https://www.researchgate.net/scientific-contributions/2049675778_Gonzalo_Mateos) page).



